In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:
#export
import sys
from os.path import join

sys.path.insert(0, '/'.join(sys.path[0].split('/')[:-1] + ['scripts']))
from learner import *

## Time to see how the model is doing

In [3]:
#export
class AvgStats():
    def __init__(self, metrics, training): 
        self.metrics = metrics
        self.training = training
    
    def reset(self):
        self.count = 0
        self.total_loss = torch.Tensor([0])
        self.totals = [torch.Tensor([0])] * len(self.metrics)
        
    @property
    def all_stats(self): return [self.total_loss] + self.totals
    
    @property
    def avg_stats(self): return [s.item()/self.count for s in self.all_stats]
    
    def __repr__(self):
        if not self.count: 
            return ''
        return f"{'train' if self.training else 'valid'} metrics - {self.avg_stats}"

    def accumulate(self, learner):
        batch_size = learner.x_batch.shape[0]
        self.count += batch_size
        self.total_loss = learner.loss * batch_size
        for i, metric in enumerate(self.metrics):
            self.totals[i] += metric(learner.pred, learner.y_batch) * batch_size

class StatsLogging(Callback):
    def __init__(self, metrics=[compute_accuracy]):
        self.train_stats = AvgStats(metrics, True)
        self.valid_stats = AvgStats(metrics, False)
        
    def before_epoch(self):
        self.train_stats.reset()
        self.valid_stats.reset()
        
    def after_loss(self):
        stats = self.train_stats if self.model.training else self.valid_stats
        stats.accumulate(self.learner)
    
    def after_epoch(self):
        print(f'Epoch - {self.epoch}\n{self.train_stats}\n{self.valid_stats}\n')

In [4]:
data_bunch = get_data_bunch(*get_mnist_data(), batch_size=64)
model = get_lin_model(data_bunch)
optimizer = DynamicOpt(list(model.parameters()), learning_rate=0.1)
loss_fn = CrossEntropy()
callbacks = [StatsLogging()]
learner = Learner(data_bunch, model, loss_fn, optimizer, callbacks)
print(learner)

(DataBunch) 
	(DataLoader) 
		(Dataset) x: (50000, 784), y: (50000,)
		(Sampler) total: 50000, batch_size: 64, shuffle: True
	(DataLoader) 
		(Dataset) x: (10000, 784), y: (10000,)
		(Sampler) total: 10000, batch_size: 128, shuffle: False
(Sequential)
	(Layer1) Linear(784, 50)
	(Layer2) ReLU()
	(Layer3) Linear(50, 10)
(CrossEntropy)
(DynamicOpt) hyper_params: ['learning_rate']
(Callbacks) ['TrainEval', 'StatsLogging']


In [5]:
learner.fit(3)s

Epoch - 1
train metrics - [0.00016816364288330078, 0.9176]
valid metrics - [6.626343727111816e-05, 0.9395]

Epoch - 2
train metrics - [6.021833419799805e-06, 0.9576]
valid metrics - [1.958775520324707e-05, 0.9628]

Epoch - 3
train metrics - [4.9700145721435544e-05, 0.96766]
valid metrics - [1.7696619033813475e-05, 0.96]



In [6]:
x_train, y_train, x_valid, y_valid = get_mnist_data()
x_train, y_train, x_valid, y_valid = x_train[:8000], y_train[:8000], x_valid[:2000], y_valid[:2000]

data_bunch = get_data_bunch(x_train, y_train, x_valid, y_valid, batch_size=64)
model = get_conv_model(data_bunch)
optimizer = DynamicOpt(list(model.parameters()), learning_rate=0.1) # dynamic optimizer
loss_fn = CrossEntropy()
callbacks = [StatsLogging()]
learner = Learner(data_bunch, model, loss_fn, optimizer, callbacks)
print(learner)

(DataBunch) 
	(DataLoader) 
		(Dataset) x: (8000, 784), y: (8000,)
		(Sampler) total: 8000, batch_size: 64, shuffle: True
	(DataLoader) 
		(Dataset) x: (2000, 784), y: (2000,)
		(Sampler) total: 2000, batch_size: 128, shuffle: False
(Sequential)
	(Layer1) Reshape(1, 28, 28)
	(Layer2) Conv2D(in: 1, out: 8, kernel: 5, stride: 4, pad: 2)
	(Layer3) ReLU()
	(Layer4) Conv2D(in: 8, out: 16, kernel: 3, stride: 2, pad: 1)
	(Layer5) Flatten()
	(Layer6) Linear(256, 10)
(CrossEntropy)
(DynamicOpt) hyper_params: ['learning_rate']
(Callbacks) ['TrainEval', 'StatsLogging']


In [7]:
learner.fit(3)

Epoch - 1
train metrics - [0.0033960378170013426, 0.805]
valid metrics - [0.025979362487792968, 0.862]

Epoch - 2
train metrics - [0.0017266731262207031, 0.908625]
valid metrics - [0.022161968231201173, 0.912]

Epoch - 3
train metrics - [0.0014250315427780152, 0.927125]
valid metrics - [0.019968509674072266, 0.908]



In [8]:
x_train, y_train, x_valid, y_valid = get_mnist_data()
x_train, y_train, x_valid, y_valid = x_train[:8000], y_train[:8000], x_valid[:2000], y_valid[:2000]

data_bunch = get_data_bunch(x_train, y_train, x_valid, y_valid, batch_size=64)
model = get_conv_final_model(data_bunch)
optimizer = DynamicOpt(list(model.parameters()), learning_rate=0.1)
loss_fn = CrossEntropy()
callbacks = [StatsLogging()]
learner = Learner(data_bunch, model, loss_fn, optimizer, callbacks)
print(learner)

(DataBunch) 
	(DataLoader) 
		(Dataset) x: (8000, 784), y: (8000,)
		(Sampler) total: 8000, batch_size: 64, shuffle: True
	(DataLoader) 
		(Dataset) x: (2000, 784), y: (2000,)
		(Sampler) total: 2000, batch_size: 128, shuffle: False
(Sequential)
	(Layer1) Reshape(1, 28, 28)
	(Layer2) Conv2D(in: 1, out: 4, kernel: 5, stride: 2, pad: 1)
	(Layer3) AvgPool2d(kernel: 2, stride: 1, pad: 0)
	(Layer4) BatchNorm()
	(Layer5) Conv2D(in: 4, out: 16, kernel: 3, stride: 2, pad: 0)
	(Layer6) BatchNorm()
	(Layer7) Flatten()
	(Layer8) Linear(400, 64)
	(Layer9) ReLU()
	(Layer10) Linear(64, 10)
(CrossEntropy)
(DynamicOpt) hyper_params: ['learning_rate']
(Callbacks) ['TrainEval', 'StatsLogging']


In [9]:
learner.fit(3)

Epoch - 1
train metrics - [0.0031955416202545167, 0.84525]
valid metrics - [0.016617767333984373, 0.901]

Epoch - 2
train metrics - [0.001386306405067444, 0.925]
valid metrics - [0.014099831581115723, 0.9225]

Epoch - 3
train metrics - [0.0022973828315734864, 0.9395]
valid metrics - [0.012099609375, 0.931]

